In [ ]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config


In [ ]:
# set up login
Cpassword = nvda_config.mysql_password
api_k = nvda_config.twelve_api_nvda
base_url = 'https://api.twelvedata.com'


In [ ]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()

# Date range for historical data (adjust as needed)
end_date = datetime.now()
start_date = end_date - timedelta(days=366)  # Get the past year's data


In [ ]:
# Twelve Data API parameters
symbol = 'NVDA'
interval = '1min'  # Adjust interval if needed (e.g., '5min', '15min', etc.)
outputsize = 5000  # Maximum number of data points per request (Twelve Data limit)

# Function to fetch data in chunks (due to Twelve Data limits)
def fetch_data_chunk(start_date, end_date):
    endpoint = f'/time_series?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)


# Function to fetch individual technical indicators in chunks
def fetch_macd_chunk(start_date, end_date):
    endpoint = f'/macd?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)

def fetch_signal_chunk(start_date, end_date):
    endpoint = f'/signal?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)

def fetch_histogram_chunk(start_date, end_date):
    endpoint = f'/histogram?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)

def fetch_ema_chunk(start_date, end_date):
    endpoint = f'/ema?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)

def fetch_rsi_chunk(start_date, end_date):
    endpoint = f'/rsi?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)

# Function to fetch technical indicators in chunks
def fetch_indicators_chunk(start_date, end_date):
    endpoint = f'/technical_indicators?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}&order=ASC&MACD=true&EMA=true&RSI=true'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)


In [ ]:


# Test fetching data and indicators to ensure no errors
try:
    

    # Test fetching MACD data
    test_macd_data = fetch_macd_chunk(start_date.strftime('%Y-%m-%d'), (start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
    if 'values' not in test_macd_data:
        raise KeyError("MACD data response does not contain 'values' key")

    # Test fetching Signal data
    test_signal_data = fetch_signal_chunk(start_date.strftime('%Y-%m-%d'), (start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
    if 'values' not in test_signal_data:
        raise KeyError("Signal data response does not contain 'values' key")

    # Test fetching Histogram data
    test_histogram_data = fetch_histogram_chunk(start_date.strftime('%Y-%m-%d'), (start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
    if 'values' not in test_histogram_data:
        raise KeyError("Histogram data response does not contain 'values' key")

    # Test fetching EMA data
    test_ema_data = fetch_ema_chunk(start_date.strftime('%Y-%m-%d'), (start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
    if 'values' not in test_ema_data:
        raise KeyError("EMA data response does not contain 'values' key")

    # Test fetching RSI data
    test_rsi_data = fetch_rsi_chunk(start_date.strftime('%Y-%m-%d'), (start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
    if 'values' not in test_rsi_data:
        raise KeyError("RSI data response does not contain 'values' key")


    # Test fetching tick data
    test_tick_data = fetch_data_chunk(start_date.strftime('%Y-%m-%d'), (start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
    if 'values' not in test_tick_data:
        raise KeyError("Tick data response does not contain 'values' key")

    # Test fetching indicator data
    test_indicator_data = fetch_indicators_chunk(start_date.strftime('%Y-%m-%d'), (start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
    if 'values' not in test_indicator_data:
        raise KeyError("Indicator data response does not contain 'values' key")

    print("Data fetching test successful. No errors found.")
except Exception as e:
    print(f"Error during data fetching test: {e}")
    raise



current_date = start_date
requests_made = 0
while current_date < end_date:
    if requests_made >= 4:  # Check if 4 requests have been made
        time.sleep(60)  # Wait for 60 seconds (1 minute)
        requests_made = 0  # Reset the request counter

    chunk_end_date = current_date + timedelta(days=30)
    tick_data = fetch_data_chunk(
        current_date.strftime("%Y-%m-%d"), chunk_end_date.strftime("%Y-%m-%d")
    )

    # Fetch technical indicators in chunks
    macd_data = fetch_macd_chunk(
        current_date.strftime("%Y-%m-%d"), chunk_end_date.strftime("%Y-%m-%d")
    )
    signal_data = fetch_signal_chunk(
        current_date.strftime("%Y-%m-%d"), chunk_end_date.strftime("%Y-%m-%d")
    )
    histogram_data = fetch_histogram_chunk(
        current_date.strftime("%Y-%m-%d"), chunk_end_date.strftime("%Y-%m-%d")
    )
    ema_data = fetch_ema_chunk(
        current_date.strftime("%Y-%m-%d"), chunk_end_date.strftime("%Y-%m-%d")
    )
    rsi_data = fetch_rsi_chunk(
        current_date.strftime("%Y-%m-%d"), chunk_end_date.strftime("%Y-%m-%d")
    )

    # Prepare for database insertion
    time_series_data = []
    indicator_data = []

    # Process and align data
    for tick in tick_data["values"]:
        datetime_str = tick["datetime"]
        matching_indicators = next(
            (
                indicator
                for indicator in macd_data["values"]
                if indicator["datetime"] == datetime_str
            ),
            None,
        )

        if matching_indicators:
            time_series_data.append(
                (
                    symbol,
                    datetime_str,
                    tick["open"],
                    tick["high"],
                    tick["low"],
                    tick["close"],
                    tick["volume"],
                )
            )
            indicator_data.append(
                (
                    symbol,
                    datetime_str,
                    matching_indicators['macd'],
                    matching_indicators['macd_signal'],
                    matching_indicators['macd_hist'],
                    next((e['ema'] for e in ema_data['values'] if e['datetime'] == datetime_str), None),
                    next((r['rsi'] for r in rsi_data['values'] if r['datetime'] == datetime_str), None),
                )
            )

    # Insert tick data into MySQL
    sql_time_series = """
        INSERT INTO stock_ticks (symbol, datetime, open, high, low, close, volume) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)

    # Insert indicator data into MySQL
    sql_indicators = """
        INSERT INTO technical_indicators (symbol, datetime, macd, signal, histogram, ema, rsi)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    mycursor.executemany(sql_indicators, indicator_data)

    current_date = chunk_end_date
    requests_made += 1

mydb.commit()
print(mycursor.rowcount, "records inserted.")